%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

In [7]:
import sys

# setting path
sys.path.append('../client')
import serving_client
import game_Client 

In [8]:
#Here I am instanciating the classes from game_client and serving_client

pinger = game_Client.gameClient(ip="127.0.0.1",port=6758)
serving = serving_client.AppClient(ip="127.0.0.1",port=6758)

In [9]:
#Input text for the model
text1 = widgets.Text(
       value="meriembchaaben",
       description='workspace',)
text2 = widgets.Text(
       value="xgboost-task5-model",
       description='model',)
text3 = widgets.Text(
       value="1.0.0",
       description='version',)

#Button1 will load the model of choice
button1 = widgets.Button(description='Load Model')

#Input text for gameID
text4 = widgets.Text(
       value='2021020329',
       description='gameID',) 

#Button2 will ping the game
button2 = widgets.Button(description='Ping Game')
out = widgets.Output()

#Here will be stored the info of games and gameID

memoryGamet1=0
memoryGamet2=0
memoryGameID=0
homeOrAway=[]
def on_button1_clicked(_):
    #This function activates when button 1 is pushed
      with out:
          clear_output()  
          #download_registry_model(self, workspace: str, model: str, version: str)
          serving.download_registry_model(str(text1.value),str(text2.value),str(text3.value))
          print("Model '",text1.value, "' Version '",text2.value,"' Workspace '",text3.value," has been uploaded")
            
def on_button2_clicked(_):
    
 
      # This function activates when button 2 is pushed 
      with out:
          clear_output()
          global memoryGamet1
          global memoryGamet2
          global memoryGameID
          global homeOrAway
            
          #Get teams and homeOrAway
          teams = list(game_Client.getTeams(str(text4.value)))

          gamet1= pinger.pingGame(teams[0],str(text4.value))        
          gamet2= pinger.pingGame(teams[1],str(text4.value))
            
          if len(gamet1)!=0 or len(gamet2)!=0:
              homeOrAway=[]
              homeOrAway.append(gamet1["homeOrAway"][1])
              homeOrAway.append(gamet2["homeOrAway"][1])
    
          #Set up memory game and ID
          if type(memoryGameID)==type(0):
              #if no memory yet, set memoryGameID
              memoryGameID=str(text4.value)
          if type(memoryGamet1)==type(0):
              #if no memory yet, set memoryGame
              memoryGamet1=gamet1
              memoryGamet2=gamet2
          else:
            if memoryGameID!=str(text4.value):
                #if the memoryGameID does not correspond to the input gameID, it means that
                #we are dealing with a new game. 
                memoryGamet1=gamet1
                memoryGamet2=gamet2
                memoryGameID=str(text4.value)
            else:
                #This is if the new events belong to the same game
                memoryGamet1.append(gamet1,ignore_index=True)
                memoryGamet2.append(gamet2,ignore_index=True)
            

            
          #Get current period and time left for that period for both dataframes
          periodt1 = memoryGamet1.iloc[-1]["period"]
          timet1 = str(memoryGamet1.iloc[-1]["periodTime"]).split(":")
          minuteLeftt1 = 19-int(timet1[0])
          secondLeftt1=60-int(timet1[1])
          timeLeftt1= str(minuteLeftt1)+":"+str(secondLeftt1)
            
          periodt2 = memoryGamet2.iloc[-1]["period"]
          timet2 = str(memoryGamet2.iloc[-1]["periodTime"]).split(":")
          minuteLeftt2 = 19-int(timet2[0])
          secondLeftt2=60-int(timet2[1])
          timeLeftt2= str(minuteLeftt2)+":"+str(secondLeftt2)
        
          #The latest value will correspond to the most recent event
          period=max([periodt1,periodt2])
          timeLeft=""
        
          if minuteLeftt2<minuteLeftt1:
              timeLeft=timeLeftt2
          elif minuteLeftt1<minuteLeftt2:
              timeLeft=timeLeftt1
          elif secondLeftt2<secondLeftt1:
              timeLeft=timeLeftt2
          elif secondLeft1<secondLeftt2:
              timeLeft=timeLeftt1
          else:
              timeLeft=timeLeftt1

                
                
          


          #Separate dataframes for each team, and get their corresponding XG

          XGt1=serving.predict(gamet1,str(text4.value),teams[0])
          XGt2=serving.predict(gamet2,str(text4.value),teams[1])

            
            
          #sum of XG
          sumt1 = sum(XGt1)
          sumt2= sum(XGt2)
        
          #Output to print 
          print("period:",period)
          print("time left:",timeLeft)
          print(homeOrAway[0],"team and expected score:",teams[0],sumt1)
          print(homeOrAway[1],"team and expected score:",teams[1],sumt2)



        

#Associate buttons to functions
button1.on_click(on_button1_clicked)
button2.on_click(on_button2_clicked)
#Widgets that will be displayed
widgets.VBox([text1,text2,text3,button1,text4,button2,out])

    def ping_game(game_id, idx, other) -> new_events, new_idx, new_other
    
    def predict(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Formats the inputs into an appropriate payload for a POST request, and queries the
        prediction service. Retrieves the response from the server, and processes it back into a
        dataframe that corresponds index-wise to the input dataframe.
        
        Args:
            X (Dataframe): Input dataframe to submit to the prediction service.
        """

        raise NotImplementedError("TODO: implement this function")

    def logs(self) -> dict:
        """Get server logs"""

        raise NotImplementedError("TODO: implement this function")

    def download_registry_model(self, workspace: str, model: str, version: str) -> dict:
        """
        Triggers a "model swap" in the service; the workspace, model, and model version are
        specified and the service looks for this model in the model registry and tries to
        download it. 

        See more here:

            https://www.comet.ml/docs/python-sdk/API/#apidownload_registry_model
        
        Args:
            workspace (str): The Comet ML workspace
            model (str): The model in the Comet ML registry to download
            version (str): The model version to download
        """

        raise NotImplementedError("TODO: implement this function")